## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sentence_transformers import SentenceTransformer, util
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import sklearn.metrics.pairwise
from simpleneighbors import SimpleNeighbors
from tqdm import tqdm
from tqdm import trange
import tensorflow.compat.v2 as tf
import torch
import time
from unidecode import unidecode
import math
import faiss

/home/chronomanteca/PythonEnv/sibea-backend/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-23 11:58:33.782303: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 11:58:36.786334: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2023-02-23 11:58:36.786421: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not

## Configuracion notebook

In [2]:
pd.set_option('display.max_columns', None)  

## Cargue de modelos

### MiniLM-L6-v2

In [2]:
#model_minilm = SentenceTransformer('all-MiniLM-L6-v2')
path = './models/model_minilm'
#model_minilm.save(path)
model_minilm = SentenceTransformer(path)

### mpnet-base-v2


In [3]:
#model_mpnet = SentenceTransformer('all-mpnet-base-v2')
path = './models/model_mpnet'
#model_mpnet.save(path)
model_mpnet = SentenceTransformer(path)

### distiluse-base-multilingual-cased-v1

In [4]:
#model_multilingual_distiluse_v1 = SentenceTransformer('distiluse-base-multilingual-cased-v1')
path = './models/model_multilingual_distiluse_v1'
#model_multilingual_distiluse_v1.save(path)
model_multilingual_distiluse_v1 = SentenceTransformer(path)

### distiluse-base-multilingual-cased-v2

In [5]:
#model_multilingual_distiluse_v2 = SentenceTransformer('distiluse-base-multilingual-cased-v2')
path = './models/model_multilingual_distiluse_v2'
#model_multilingual_distiluse_v2.save(path)
model_multilingual_distiluse_v2 = SentenceTransformer(path)

### paraphrase-multilingual-MiniLM-L12-v2

In [6]:
#model_multilingual_minilm = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
path = './models/model_multilingual_minilm'
#model_multilingual_minilm.save(path)
model_multilingual_minilm = SentenceTransformer(path)

### paraphrase-multilingual-mpnet-base-v2

In [7]:
#model_multilingual_mpnet = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
path = './models/model_multilingual_mpnet'
#model_multilingual_mpnet.save(path)
model_multilingual_mpnet = SentenceTransformer(path)


## Gestion de rutas y lista de modelos

In [16]:
import os


In [18]:

this_dir = os.path.abspath('')

In [217]:
models = [
        'model_minilm',
        'model_mpnet',
        'model_multilingual_distiluse_v1',
        'model_multilingual_distiluse_v2',
        'model_multilingual_minilm',
        'model_multilingual_mpnet']

## Generación de embeddings (Normalizados)

In [35]:
def embed(messages,model):
    if model=='model_minilm':
        path = './models/model_minilm'
        model = SentenceTransformer(path) 
        return model.encode(messages,normalize_embeddings=True)
       # wanted_dir = os.path.join(this_dir,path)
        #print(wanted_dir)
    elif model == 'model_mpnet':
        path = './models/model_mpnet'
        model = SentenceTransformer(path) 
        return model.encode(messages,normalize_embeddings=True) 
       # wanted_dir = os.path.join(this_dir,path)
    elif model == 'model_multilingual_distiluse_v1':
        path = './models/model_multilingual_distiluse_v1'
        model = SentenceTransformer(path) 
        return model.encode(messages,normalize_embeddings=True) 
        #wanted_dir = os.path.join(this_dir,path)
    elif model == 'model_multilingual_distiluse_v2':
        path = './models/model_multilingual_distiluse_v2'
        model = SentenceTransformer(path) 
        return model.encode(messages,normalize_embeddings=True)
       #wanted_dir = os.path.join(this_dir,path)
    elif model == 'model_multilingual_minilm':
        path = './models/model_multilingual_minilm'
        model = SentenceTransformer(path) 
        return model.encode(messages,normalize_embeddings=True)
        #wanted_dir = os.path.join(this_dir,path)
    elif model == 'model_multilingual_mpnet':
        path = './models/model_multilingual_mpnet'
        model = SentenceTransformer(path) 
        return model.encode(messages,normalize_embeddings=True)
        #wanted_dir = os.path.join(this_dir,path)
    
    

In [22]:
def append_embeddings(messages,model):   
    print("generando embeddings para el modelo "+model)     
    results = []
    
    #df[["titulo_del_proyecto","nombre_facultad","nombre_del_departamento","descripcion","resumen","objetivos","metodologia","gran_area","objetivo_socioeconomico","palabras_clave"]]
    i = 0
    for row in messages.iterrows():
        embedString = messages.at[row,"corpus"]
       # print(embedString)
        embedding = embed(embedString,model)
        print("Documentos vectorizados: "+str(i))
        i=i+1
        results.append({"titulo":row["titulo_del_proyecto"],"facultad":row["nombre_facultad"],"departamento":row["nombre_del_departamento"],"resumen":row["resumen"],"objetivos":row["objetivos"],"palabras_clave":row["palabras_clave"],"vector":embedding.tolist()})
    return results

## Procesamiento del Dataframe

In [187]:
def get_dataframe():
    print("procesando datos...")
    
    inv_df = pd.read_excel(os.path.join(this_dir,'./xlsx/InvestigarPUJ.xlsx'), sheet_name='Hoja1', converters={'ID PROYECTO':str})
    siap_df = pd.read_excel(os.path.join(this_dir,'./xlsx/Descriptores SIAP 2023.xlsx'), sheet_name='SIAP ', converters={'ID Proy':str})
    inv_full_df = pd.read_excel(os.path.join(this_dir,'./xlsx/Descriptores SIAP 2023.xlsx'), sheet_name='InvestigarPUJ', converters={'Id':str})

    print(inv_df.columns)

    inv_full_df.drop("Año", axis=1, inplace=True)

    datasets = [inv_df, siap_df, inv_full_df]
    for dataset in datasets:
        dataset.replace('\\N', np.NaN, inplace=True)
        dataset.replace('null', np.NaN, inplace=True)
        dataset.replace('nan', np.NaN, inplace=True)
        dataset.replace('N/A', np.NaN, inplace=True)


    merged = inv_df.merge(siap_df, left_on="ID PROYECTO", right_on="ID Proy", how="left")
    df = merged.merge(inv_full_df, left_on="ID PROYECTO", right_on="Id", how="left")

    df = df.groupby("ID PROYECTO").agg(lambda x: list(set(x))).applymap(lambda x: ', '.join(str(y) for y in x if str(y) != 'nan') if isinstance(x, list) else x)
    df = df.reset_index()
    df.columns = [unidecode(x.lower().strip().replace(" ", "_").replace("__", "_").replace(".", "")) for x in df.columns]
    print(df.columns)

    df.replace('', np.NaN, inplace=True)

    df["descripcion"] = df["descripcion_x"].fillna("") + df["descripcion_y"].fillna("")

    df.drop("titulo_x", axis=1, inplace=True)
    df.drop("titulo_y", axis=1, inplace=True)
    df.drop("id_proy", axis=1, inplace=True)
    df.drop("id", axis=1, inplace=True)
    df.drop("financiador", axis=1, inplace=True)
    df.drop("tipo_propuesta", axis=1, inplace=True)
    df.drop("descripcion_x", axis=1, inplace=True)
    df.drop("descripcion_y", axis=1, inplace=True)
    df.drop("id_propt", axis=1, inplace=True)
    df.drop("f_inic_real", axis=1, inplace=True)
    df.drop("f_final_real", axis=1, inplace=True)
    df.drop("facultad", axis=1, inplace=True)
    df.drop("departamento", axis=1, inplace=True)
    df.drop("estado_proyecto", axis=1, inplace=True)
    df.drop("nombre", axis=1, inplace=True)
    df.drop("cantidad", axis=1, inplace=True)
    df.drop("fecha_de_negociacion", axis=1, inplace=True)
    df.drop("fecha_inicial_real", axis=1, inplace=True)
    df.drop("fecha_final_real", axis=1, inplace=True)
    df.drop("convocatoria", axis=1, inplace=True)
    df.drop("miembro_del_equipo", axis=1, inplace=True)
    #print(df.columns)
    #print(df.shape)

    #df[df["id_proyecto"] == "000000000007161"]
    #df[df["id_proyecto"] == "004438"]
    #df[df["id_proyecto"] == "20104"]
    df["corpus"] = df["titulo_del_proyecto"].fillna("") + " "  + \
    df["nombre_facultad"].str.split().str[-1].fillna("") + " " + \
    df["nombre_del_departamento"].str.split().str[-1].fillna("") + " " + \
    df["descripcion"].fillna("") + " " + \
    df["resumen"].fillna("") + " " + \
    df["objetivos"].fillna("") + " " + \
    df["metodologia"].fillna("") + " " + \
    df["gran_area"].fillna("") + " " + \
    df["objetivo_socioeconomico"].fillna("") + " " + \
    df["palabras_clave"].fillna("")
    newdf = df[["corpus","titulo_del_proyecto","nombre_facultad","nombre_del_departamento","descripcion","resumen","objetivos","metodologia","gran_area","objetivo_socioeconomico","palabras_clave"]]


    #df["corpus"] = df["corpus"].replace(r'\n',' ', regex=True).str.strip()

   # df.to_csv("./csv/prelimpieza.csv", index=False)
    #newdf.to_csv("./csv/limpieza.csv", index=False)
    return df

In [ ]:
inv_df = pd.read_excel(os.path.join(this_dir,'./xlsx/InvestigarPUJ.xlsx'), sheet_name='Hoja1', converters={'ID PROYECTO':str})
siap_df = pd.read_excel(os.path.join(this_dir,'./xlsx/Descriptores SIAP 2023.xlsx'), sheet_name='SIAP ', converters={'ID Proy':str})
inv_full_df = pd.read_excel(os.path.join(this_dir,'./xlsx/Descriptores SIAP 2023.xlsx'), sheet_name='InvestigarPUJ', converters={'Id':str})



In [132]:
print(inv_df.columns)

Index(['ID_PROPUESTA', 'TIPO DE PROPUESTA', 'FECHA DE NEGOCIACION', 'AÑO',
       'ID PROYECTO', 'TITULO DEL PROYECTO', 'FECHA INICIAL REAL',
       'FECHA FINAL REAL', 'NOMBRE FACULTAD', 'NOMBRE DEL DEPARTAMENTO',
       'CONVOCATORIA', 'NOMBRE PATROCINADOR', 'NOMBRE GRUPO DE INVESTIGACION',
       'MIEMBRO DEL EQUIPO', 'TIPO DE DOCUMENTO', 'NUMERO DE DOCUMENTO',
       'NOMBRES Y APELLIDOS', 'ROL EN EL PROYECTO', 'PORCENTAJE DEDICACION',
       'CÓDIGO PRESUPUESTO', 'CÓDIGO CONTRATO SECRE JURÍDICA',
       'VALOR APROBADO PATROCINADOR', 'CONTRAPARTIDA TERCEROS',
       'CONTRAPARTIDA REC. PROPI', 'CONTRAPARTIDA REC. NUEVO',
       'REC. NUEVOS UNIDADE ACADEMICA', 'VALOR CONTRAPARTIDA TOTAL',
       'VALOR TOTAL', 'ESTADO '],
      dtype='object')


In [142]:
print(inv_df[inv_df['TIPO DE PROPUESTA']=='AJI'])

      ID_PROPUESTA TIPO DE PROPUESTA FECHA DE NEGOCIACION   AÑO  \
778       00005203               AJI           2012-11-08  2012   
779       00005203               AJI           2012-11-08  2012   
813       00005214               AJI           2012-11-08  2012   
814       00005214               AJI           2012-11-08  2012   
815       00005215               AJI           2012-11-08  2012   
...            ...               ...                  ...   ...   
14365     00009921               AJI           2021-11-09  2021   
14366     00009921               AJI           2021-11-09  2021   
14385     00009928               AJI           2021-11-09  2021   
14386     00009928               AJI           2021-11-09  2021   
14387     00009928               AJI           2021-11-09  2021   

           ID PROYECTO                 TITULO DEL PROYECTO FECHA INICIAL REAL  \
778             005080       Paola Andrea Hernández Suárez         2013-05-15   
779             005080       Paol

In [133]:
print(siap_df.columns)

Index(['ID Propt', 'ID Proy', 'Tipo  Propuesta', 'Título', 'Financiador',
       'F Inic Real', 'F Final Real', 'Facultad', 'Departamento',
       'Estado proyecto', 'Descripción', 'Código Tipo', 'Nombre'],
      dtype='object')


In [134]:
print(inv_full_df.columns)

Index(['Id', 'Convocatoria nombre corto', 'Convocatoria', 'Tipo de actividad',
       'Título', 'Fecha estimada inicio', 'Fecha estimada fin',
       'Id empleado Responsable PUJ', 'Nombre Responsable PUJ',
       'Departamento Responsable PUJ', 'Id departamento Responsable PUJ',
       'Facultad Responsable PUJ', 'Nombre financiador principal', 'Resumen',
       'Objetivos', 'Metodología', 'Categoría', 'Producto', 'Cantidad',
       'Descripción', 'Valor solicitado a financiador principal',
       'Valor total financiadores', 'Tipo de investigación', 'Gran área',
       'Objetivo Socioeconómico', 'Palabras Clave', 'Aprobado?', 'Año'],
      dtype='object')


In [188]:
testdf = get_dataframe()


procesando datos...
Index(['ID_PROPUESTA', 'TIPO DE PROPUESTA', 'FECHA DE NEGOCIACION', 'AÑO',
       'ID PROYECTO', 'TITULO DEL PROYECTO', 'FECHA INICIAL REAL',
       'FECHA FINAL REAL', 'NOMBRE FACULTAD', 'NOMBRE DEL DEPARTAMENTO',
       'CONVOCATORIA', 'NOMBRE PATROCINADOR', 'NOMBRE GRUPO DE INVESTIGACION',
       'MIEMBRO DEL EQUIPO', 'TIPO DE DOCUMENTO', 'NUMERO DE DOCUMENTO',
       'NOMBRES Y APELLIDOS', 'ROL EN EL PROYECTO', 'PORCENTAJE DEDICACION',
       'CÓDIGO PRESUPUESTO', 'CÓDIGO CONTRATO SECRE JURÍDICA',
       'VALOR APROBADO PATROCINADOR', 'CONTRAPARTIDA TERCEROS',
       'CONTRAPARTIDA REC. PROPI', 'CONTRAPARTIDA REC. NUEVO',
       'REC. NUEVOS UNIDADE ACADEMICA', 'VALOR CONTRAPARTIDA TOTAL',
       'VALOR TOTAL', 'ESTADO '],
      dtype='object')
Index(['id_proyecto', 'id_propuesta', 'tipo_de_propuesta',
       'fecha_de_negociacion', 'ano', 'titulo_del_proyecto',
       'fecha_inicial_real', 'fecha_final_real', 'nombre_facultad',
       'nombre_del_departamento',

### further cleaning

In [171]:
print(testdf.columns)

Index(['id_proyecto', 'id_propuesta', 'tipo_de_propuesta', 'ano',
       'titulo_del_proyecto', 'nombre_facultad', 'nombre_del_departamento',
       'nombre_patrocinador', 'nombre_grupo_de_investigacion',
       'tipo_de_documento', 'numero_de_documento', 'nombres_y_apellidos',
       'rol_en_el_proyecto', 'porcentaje_dedicacion', 'codigo_presupuesto',
       'codigo_contrato_secre_juridica', 'valor_aprobado_patrocinador',
       'contrapartida_terceros', 'contrapartida_rec_propi',
       'contrapartida_rec_nuevo', 'rec_nuevos_unidade_academica',
       'valor_contrapartida_total', 'valor_total', 'estado', 'codigo_tipo',
       'convocatoria_nombre_corto', 'tipo_de_actividad',
       'fecha_estimada_inicio', 'fecha_estimada_fin',
       'id_empleado_responsable_puj', 'nombre_responsable_puj',
       'departamento_responsable_puj', 'id_departamento_responsable_puj',
       'facultad_responsable_puj', 'nombre_financiador_principal', 'resumen',
       'objetivos', 'metodologia', 'categori

In [189]:
print(len(testdf))

2853


In [190]:
##dropping fields
list_columns_drop = ['ano',
                     'nombre_patrocinador',
                     'tipo_de_documento',
                     'numero_de_documento',
                     'nombres_y_apellidos',
                     'rol_en_el_proyecto',
                     'porcentaje_dedicacion',
                     'codigo_presupuesto',
                     'codigo_contrato_secre_juridica',
                     'valor_aprobado_patrocinador',
                     'contrapartida_terceros',
                     'contrapartida_rec_propi',
                     'contrapartida_rec_nuevo',
                     'rec_nuevos_unidade_academica',
                     'valor_contrapartida_total',
                     'valor_total',
                     'fecha_estimada_inicio',
                     'fecha_estimada_fin',
                     'id_empleado_responsable_puj',
                     'nombre_responsable_puj',
                     'departamento_responsable_puj',
                     'id_departamento_responsable_puj',
                     'facultad_responsable_puj',
                     'producto',
                     'valor_solicitado_a_financiador_principal',
                     'valor_total_financiadores',
                     'tipo_de_investigacion'            
]

testdf = testdf.drop(list_columns_drop,axis = 1)


print(testdf.columns)

Index(['id_proyecto', 'id_propuesta', 'tipo_de_propuesta',
       'titulo_del_proyecto', 'nombre_facultad', 'nombre_del_departamento',
       'nombre_grupo_de_investigacion', 'estado', 'codigo_tipo',
       'convocatoria_nombre_corto', 'tipo_de_actividad',
       'nombre_financiador_principal', 'resumen', 'objetivos', 'metodologia',
       'categoria', 'gran_area', 'objetivo_socioeconomico', 'palabras_clave',
       'aprobado?', 'descripcion', 'corpus'],
      dtype='object')


In [192]:
#Removing AJI value from dataset
testdf = testdf[testdf['tipo_de_propuesta']!='AJI']
print(testdf[testdf['tipo_de_propuesta']=='AJI'])

Empty DataFrame
Columns: [id_proyecto, id_propuesta, tipo_de_propuesta, titulo_del_proyecto, nombre_facultad, nombre_del_departamento, nombre_grupo_de_investigacion, estado, codigo_tipo, convocatoria_nombre_corto, tipo_de_actividad, nombre_financiador_principal, resumen, objetivos, metodologia, categoria, gran_area, objetivo_socioeconomico, palabras_clave, aprobado?, descripcion, corpus]
Index: []

[0 rows x 22 columns]


In [193]:
print(len(testdf))

2717


In [175]:
print(testdf.columns)

Index(['id_proyecto', 'id_propuesta', 'tipo_de_propuesta',
       'titulo_del_proyecto', 'nombre_facultad', 'nombre_del_departamento',
       'nombre_grupo_de_investigacion', 'estado', 'codigo_tipo',
       'convocatoria_nombre_corto', 'tipo_de_actividad',
       'nombre_financiador_principal', 'resumen', 'objetivos', 'metodologia',
       'categoria', 'gran_area', 'objetivo_socioeconomico', 'palabras_clave',
       'aprobado?', 'descripcion', 'corpus'],
      dtype='object')


In [215]:
for row,index in testdf.iterrows():
    testdf.at[row,"corpus"] = unidecode(testdf.at[row,"corpus"])
    testdf.at[row,"titulo_del_proyecto"] = unidecode(testdf.at[row,"titulo_del_proyecto"])
    testdf.at[row,"nombre_facultad"] = unidecode(testdf.at[row,"nombre_facultad"])
    testdf.at[row,"nombre_del_departamento"] = unidecode(testdf.at[row,"nombre_del_departamento"])
    testdf.at[row,"nombre_grupo_de_investigacion"] = unidecode(str(testdf.at[row,"nombre_grupo_de_investigacion"]))
    testdf.at[row,"estado"] = unidecode(str(testdf.at[row,"estado"]))
    testdf.at[row,"tipo_de_actividad"] = unidecode(str(testdf.at[row,"tipo_de_actividad"]))
    testdf.at[row,"tipo_de_propuesta"] = unidecode(str(testdf.at[row,"tipo_de_propuesta"]))
    testdf.at[row,"nombre_financiador_principal"] = unidecode(str(testdf.at[row,"nombre_financiador_principal"]))
    testdf.at[row,"resumen"] = unidecode(str(testdf.at[row,"resumen"]))
    testdf.at[row,"objetivos"] = unidecode(str(testdf.at[row,"objetivos"]))
    testdf.at[row,"metodologia"] = unidecode(str(testdf.at[row,"metodologia"]))
    testdf.at[row,"gran_area"] = unidecode(str(testdf.at[row,"gran_area"]))
    testdf.at[row,"objetivo_socioeconomico"] = unidecode(str(testdf.at[row,"objetivo_socioeconomico"]))
    testdf.at[row,"palabras_clave"] = unidecode(str(testdf.at[row,"palabras_clave"]))
    testdf.at[row,"descripcion"] = unidecode(str(testdf.at[row,"descripcion"]))
    


In [204]:
for row,index in testdf.iterrows():
    print(testdf.at[row,"descripcion"])

Young-onset neurodegenerative diseases such as frontotemporal degeneration (FTD) and early-onset Alzheimer?s Disease (AD) affect individuals in their productive years and represent a major source of disability. In addition to their impact on health services and care, dementing illnesses produce social, emotional and financial stress for patients and their families, ASSESSMENT OF RESEARCH AND PATIENT NEEDS IN COLOMBIA. During the initial stage of the project, we will estimate the prevalence and incidence of young-onset dementias in Colombia by evaluating the patient population at the largest dementia research center in Colombia (MCIA).  
DEVELOP COLLABORATIONS AND RESOURCES NEEDED FOR THE STUDY OF YOUNG-ONSET DEMENTIAS.  We will identify key personnel at the MCIA and pair them with PTDC experts in the fields of neurology, neuropsychology, genetics and neur
1. Caracterizar el tipo de practicas de religiosidad popular que se llevan a cabo en el Parque Zonal Villa Mayor. 
2. Identificar lo

In [24]:
import json

In [39]:
 
def embeddings2json(model,df):       
    dict = append_embeddings(df,model)
    print("generando archivo JSON para el modelo "+model)    
    j = json.dumps(dict)    
    jsonFile = open(os.path.join(this_dir,"./json/"+model+".json"), "w")
    jsonFile.write(j)
    jsonFile.close()
    print(json)
    return json

### Agregando tensores al nuevo df y guardandolo como JSON

In [216]:
print(testdf[testdf['resumen']=='nan'])

          id_proyecto id_propuesta                    tipo_de_propuesta  \
0     000000000005531     00005566                                  NEW   
1     000000000005532     00005567                                  NEW   
2     000000000005533     00005568                                  NEW   
3     000000000005536     00005571                                  ANP   
4     000000000005537     00005572                                  NEW   
...               ...          ...                                  ...   
2825            20624        20624           Semillero de investigacion   
2826            20625        20625           Semillero de investigacion   
2850            20668        20668  Apoyo a la formulacion de proyectos   
2851            20670        20670           Semillero de investigacion   
2852            20671        20671           Semillero de investigacion   

                                    titulo_del_proyecto  \
0                     Young-onset dement

In [218]:
for m in models:
    embeddings2json(m,testdf)

generando embeddings para el modelo model_minilm
Documentos vectorizados: 0
Documentos vectorizados: 1
Documentos vectorizados: 2
Documentos vectorizados: 3
Documentos vectorizados: 4
Documentos vectorizados: 5
Documentos vectorizados: 6
Documentos vectorizados: 7
Documentos vectorizados: 8
Documentos vectorizados: 9
Documentos vectorizados: 10
Documentos vectorizados: 11
Documentos vectorizados: 12
Documentos vectorizados: 13
Documentos vectorizados: 14
Documentos vectorizados: 15
Documentos vectorizados: 16
Documentos vectorizados: 17
Documentos vectorizados: 18
Documentos vectorizados: 19
Documentos vectorizados: 20
Documentos vectorizados: 21
Documentos vectorizados: 22
Documentos vectorizados: 23
Documentos vectorizados: 24
Documentos vectorizados: 25
Documentos vectorizados: 26
Documentos vectorizados: 27
Documentos vectorizados: 28
Documentos vectorizados: 29
Documentos vectorizados: 30
Documentos vectorizados: 31
Documentos vectorizados: 32
Documentos vectorizados: 33
Documento

In [33]:
categories = df["nombre_facultad"].groupby(df["nombre_facultad"]).count().sort_values(ascending=False)
top20 = df["titulo_del_proyecto"].drop_duplicates().str.split(expand=True).stack().value_counts()[:20]

### Test

In [36]:
messages = [
    "I like my phone",
    "My cellphone is not good.",
    "I'm going to call someone with my phone",
    "Will it snow tomorrow?",
    "Recently a lot of hurricanes have hit the US",
    "Global warming is real",
    "An apple a day, keeps the doctors away",
    "Eating strawberries is healthy",
    "Is paleo better than keto?",
    "Healthcare system",
    "COVID-19",
    "ADHD",
    "CPAP",
    "Lungs",
    "Psychology",
    "BMI",
    "Fat"
]
for model in models:
    print(model)
    embeddings = embed(messages,models)

model_minilm
model_mpnet
model_multilingual_distiluse_v1
model_multilingual_distiluse_v2
model_multilingual_minilm
model_multilingual_mpnet


### Guardando tensores generados

In [10]:
minilm_tensor = embeddings[0].get('tensor')
mpnet_tensor = embeddings[1].get('tensor')
multilingual_distiluse_v1_tensor = embeddings[2].get('tensor')
multilingual_distiluse_v2_tensor = embeddings[3].get('tensor')
multilingual_minilm_tensor = embeddings[4].get('tensor')

np.save('./tensors/binary/minilm_tensor.npy', minilm_tensor)
np.save('./tensors/binary/mpnet_tensor.npy', mpnet_tensor)
np.save('./tensors/binary/multilingual_distiluse_v1_tensor.npy', multilingual_distiluse_v1_tensor)
np.save('./tensors/binary/multilingual_distiluse_v2_tensor.npy', multilingual_distiluse_v2_tensor)
np.save('./tensors/binary/multilingual_minilm_tensor.npy', multilingual_minilm_tensor)

### Cargando tensores generados

In [ ]:
embeddings = []
embeddings.append({"model":"minilm","tensor":np.load('./tensor/binary/minilm_tensor.npy')})
embeddings.append({"model":"mpnet","tensor":np.load('./tensor/binary/mpnet_tensor.npy')})
embeddings.append({"model":"multilingual_distiluse_v1","tensor":np.load('./tensor/binary/multilingual_distiluse_v1_tensor.npy')})
embeddings.append({"model":"multilingual_distiluse_v2","tensor":np.load('./tensor/binary/multilingual_distiluse_v2_tensor.npy')})
embeddings.append({"model":"multilingual_minilm","tensor":np.load('./tensor/binary/multilingual_minilm_tensor.npy')})
print(embeddings)

### Guardando como CSV

In [18]:
np.savetxt('./tensors/csv/minilm_tensor.csv', minilm_tensor,delimiter=",")
np.savetxt('./tensors/csv/mpnet_tensor.csv', mpnet_tensor,delimiter=",")
np.savetxt('./tensors/csv/multilingual_distiluse_v1_tensor.csv', multilingual_distiluse_v1_tensor,delimiter=",")
np.savetxt('./tensors/csv/multilingual_distiluse_v2_tensor.csv', multilingual_distiluse_v2_tensor,delimiter=",")
np.savetxt('./tensors/csv/multilingual_minilm_tensor.csv', multilingual_minilm_tensor,delimiter=",")

In [42]:
arr = embeddings[0].get("tensor")
print(arr.shape)
arr = embeddings[1].get("tensor")
print(arr.shape)
arr = embeddings[2].get("tensor")
print(arr.shape)
arr = embeddings[3].get("tensor")
print(arr.shape)
arr = embeddings[4].get("tensor")
print(arr.shape)
arr = embeddings[5].get("tensor")
print(arr.shape)

(1000, 384)
(1000, 768)
(1000, 512)
(1000, 512)
(1000, 384)


IndexError: list index out of range

In [ ]:
plot_embeddings_timing(embeddings, "time_encoding")
plot_embeddings_timing(embeddings, "time_indexing")

### Intento de guardar los embeddings en disco


In [18]:
embeddings = np.loadtxt('embeddings.csv', delimiter=',')
"""
print(type(embeddings[0]))
print(type(embeddings[0].get('tensor')))
f = open('embeddings.csv', 'a')
for embedding in embeddings:
    for key in embedding:
        #print(embedding.get('tensor'))
        f.write("\n")
        np.savetxt(f,embedding.get('tensor'),delimiter=',')
"""






ValueError: the number of columns changed from 384 to 768 at row 4001; use `usecols` to select a subset and avoid this error

### Prueba busqueda

In [ ]:
results = search_all("Explotacion infantil", 10, embeddings)
plot_search_timing(results)
search_results(results, messages, 2)